### Importing libraries

In [29]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error,mean_squared_error,mean_absolute_error

### Importing Datasets

In [30]:
khm_so2=pd.read_excel(r'khammam_so2.xlsx')
khm_nox=pd.read_excel(r'khammam_no2.xlsx')
khm_pm10=pd.read_excel(r'khammam_PM10.xlsx')
khm_aqi=pd.read_excel(r'khammam_aqi.xlsx')

### Filling the null values row-wise as there seems to more yearly relation than monthly relation for each pollutant

In [31]:
for i in range(0,7):
    khm_so2.iloc[i,1:]=khm_so2.iloc[i,1:].fillna(khm_so2.iloc[i,1:].mean())
    khm_nox.iloc[i,1:]=khm_nox.iloc[i,1:].fillna(khm_nox.iloc[i,1:].mean())
    khm_pm10.iloc[i,1:]=khm_pm10.iloc[i,1:].fillna(khm_pm10.iloc[i,1:].mean())
    khm_aqi.iloc[i,1:]=khm_aqi.iloc[i,1:].fillna(khm_aqi.iloc[i,1:].mean())

### Reorganizing the data for a combined dataset.

In [32]:
khm_so2 = khm_so2.melt(id_vars=["Year"], var_name="Month", value_name="SO2")
khm_nox = khm_nox.melt(id_vars=["Year"], var_name="Month", value_name="NOX")
khm_pm10 = khm_pm10.melt(id_vars=["Year"], var_name="Month", value_name="PM10")
khm_aqi = khm_aqi.melt(id_vars=["Year"], var_name="Month", value_name="AQI")

### Organizing the data for better model usage

In [33]:
khm_poll = pd.merge(khm_pm10, khm_nox, on=['Month','Year'])
khm_poll = pd.merge(khm_poll, khm_so2, on=['Month','Year'])
khm_final=pd.merge(khm_poll,khm_aqi,on=['Month','Year'])
khm_final.index = pd.to_datetime(khm_final['Year'].astype(str) + '-' + khm_final['Month'], format='%Y-%b')
khm_final.drop(['Year', 'Month'], axis=1, inplace=True)
khm_final=khm_final.sort_index()
khm_final['2016':'2021']

,PM10,NOX,SO2,AQI
2016-01-01,46.0,19.0,6.0,46.666667
2016-02-01,51.0,19.0,7.0,51.333333
2016-03-01,54.0,20.7,6.1,53.888889
2016-04-01,51.0,19.0,8.0,50.814815
2016-05-01,47.0,19.0,7.0,47.444444
...,...,...,...,...
2021-08-01,67.0,35.9,6.0,67.000000
2021-09-01,85.0,34.0,9.2,84.938737
2021-10-01,61.0,31.0,7.4,61.000000
2021-11-01,86.0,37.6,7.5,86.000000


### Test Train Split

In [34]:
X_=khm_final.iloc[:,:-1]
y=khm_final.iloc[:,-1]

In [35]:
#Scalling the Data to improve model perfomance and bring features into similar range
ss=StandardScaler()
X=ss.fit_transform(X_)
X=pd.DataFrame(X,columns=khm_final.columns[:-1])

In [36]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=66,test_size=0.25)

### Perfoming Hyper-parameter tuning with comparison with other models

In [37]:
#Listing Parameters

rf_params = {'n_estimators': [500,300,100,800,1000], 'max_depth': [12,15,10,5,7,3]}

lr_param_grid = {}

svr_param_grid = {'kernel': ['linear', 'rbf'], 'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10]}

In [38]:
# create models
rf_model = RandomForestRegressor(random_state=42)
lr_model = LinearRegression()
svr_model = SVR()

In [39]:
# create GridSearchCV objects
rf_grid = GridSearchCV(rf_model, rf_params, cv=5)
lr_grid = GridSearchCV(lr_model, lr_param_grid)
svr_grid = GridSearchCV(svr_model, svr_param_grid)

In [40]:
# fit the models
rf_grid.fit(X_train, y_train)
lr_grid.fit(X_train, y_train)
svr_grid.fit(X_train, y_train)

GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10],
                         'kernel': ['linear', 'rbf']})

In [41]:
# get the best hyperparameters
rf_best_params = rf_grid.best_params_
lr_best_params = lr_grid.best_params_
svr_best_params = svr_grid.best_params_

In [42]:
# create new models with the best hyperparameters
rf_best_model = RandomForestRegressor(**rf_best_params, random_state=42)
lr_best_model = LinearRegression(**lr_best_params)
svr_best_model = SVR(**svr_best_params)

In [43]:
# fit the best models
rf_best_model.fit(X_train, y_train)
lr_best_model.fit(X_train, y_train)
svr_best_model.fit(X_train, y_train)

SVR(C=1, gamma=0.1, kernel='linear')

In [44]:
# make predictions
rf_preds = rf_best_model.predict(X_test)
lr_preds = lr_best_model.predict(X_test)
svr_preds = svr_best_model.predict(X_test)

### Checking individual model performace

In [45]:
# calculate r2 scores

rf_r2 = r2_score(y_test, rf_preds)
print(f"Random Forest R2 Score: {rf_r2}")

lr_r2 = r2_score(y_test, lr_preds)
print(f"Linear Regression R2 Score: {lr_r2}")

svr_r2 = r2_score(y_test, svr_preds)
print(f"Support Vector Regressor R2 Score: {svr_r2}")

Random Forest R2 Score: 0.9363947935075052
Linear Regression R2 Score: 0.9601112131975434
Support Vector Regressor R2 Score: 0.9775007671278183


### Checking Ridge and Lasso Regression and cross validation to prevent overfitting

In [46]:
lr = LinearRegression()
ridge = Ridge()
lasso = Lasso()
rf = RandomForestRegressor()
svr = SVR()

lr_params = {'normalize': [True, False]}
ridge_params = {'alpha': [0.01, 0.1, 1, 10, 100]}
lasso_params = {'alpha': [0.01, 0.1, 1, 10, 100]}
rf_params = {'n_estimators': [500,300,100,800,1000], 'max_depth':[12,15,10,5,7,3] }
svr_params = {'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}

In [47]:
models = {'Linear Regression': (lr, lr_params),
          'Ridge Regression': (ridge, ridge_params),
          'Lasso Regression': (lasso, lasso_params),
          'Random Forest Regression': (rf, rf_params),
          'Support Vector Regression': (svr, svr_params)}

In [48]:
models = {'Linear Regression': (lr, lr_params),
          'Ridge Regression': (ridge, ridge_params),
          'Lasso Regression': (lasso, lasso_params),
          'Random Forest Regression': (rf, rf_params),
          'Support Vector Regression': (svr, svr_params)}
for name, (model, params) in models.items():
    gs = GridSearchCV(model, params, cv=2,n_jobs=-1)
    scores = cross_val_score(gs, X_test, y_test, cv=2, scoring='r2')
    print(f'{name}: {scores.mean():.3f} (±{scores.std():.3f})')

Linear Regression: 0.978 (±0.000)
Ridge Regression: 0.960 (±0.018)
Lasso Regression: 0.976 (±0.002)
Random Forest Regression: 0.760 (±0.078)
Support Vector Regression: 0.987 (±0.005)


### Performing ensemble methods and full model accuracy

In [49]:
import numpy as np
f_pred=[]
f_pred.append((rf_preds+lr_preds+svr_preds)/3)
f_pred=np.array(f_pred)
f_pred=f_pred.reshape(-1,1)

In [50]:
rr2 = r2_score(y_test, f_pred)
print(f" R2 Score: {rr2}")

mape = mean_absolute_percentage_error(y_test, f_pred)
print(f" Mean Absolute Percentage Error: {mape}")

mse=mean_squared_error(y_test,f_pred)
print(f"Mean Squared Error : {mape}")


 R2 Score: 0.9632557016736956
 Mean Absolute Percentage Error: 0.03284906852676318
Mean Squared Error : 0.03284906852676318


### Final Predictions


In [51]:
loc=[]
for i in f_pred:
    if (i>=0) and i<=50:
        loc.append('Good')
    elif i>=51 and i<=100:
        loc.append('Moderate')
    elif i>=101 and i<=150:
        loc.append('Unhealthy for Sensitive Groups')
    elif i>=151 and i<=200:
        loc.append('Unhealthy')
    elif i>=201 and i<=300:
        loc.append('Very Unhealthy')
    else:
        loc.append('Hazardous')
y_test=pd.DataFrame(y_test)
y_test['Level of Concern']=loc

In [52]:
y_test

,AQI,Level of Concern
2019-08-01,84.083333,Moderate
2021-09-01,84.938737,Moderate
2017-04-01,54.555556,Moderate
2018-10-01,84.697653,Moderate
2019-11-01,84.851852,Moderate
2020-01-01,88.296296,Moderate
2021-11-01,86.000000,Moderate
2020-05-01,63.000000,Moderate
2022-09-01,42.000000,Good
2022-02-01,78.079300,Moderate


### Predictor function

In [53]:
def predictor(df):
    df=ss.transform(df.values)
    rfp=rf_best_model.predict(df)
    lrp=lr_best_model.predict(df)
    svrp=svr_best_model.predict(df)
    f_pred=[]
    f_pred.append((rfp+lrp+svrp)/3)
    f_pred=np.array(f_pred)
    f_pred=f_pred.reshape(-1,1)
    loc=[]
    for i in f_pred:
        if (i>=0) and i<=50:
            loc.append('Good')
        elif i>=51 and i<=100:
            loc.append('Moderate')
        elif i>=101 and i<=150:
            loc.append('Unhealthy for Sensitive Groups')
        elif i>=151 and i<=200:
            loc.append('Unhealthy')
        elif i>=201 and i<=300:
            loc.append('Very Unhealthy')
        else:
            loc.append('Hazardous')
    df_p=pd.DataFrame(f_pred,columns=['AQI'])
    df_p['AQI']=f_pred
    df_p['Level of Concern']=np.array(loc).reshape(-1,1)
    print(df_p)
    return df_p

In [54]:

res=predictor(X_['2022'])

           AQI Level of Concern
0    77.935181         Moderate
1    76.597783         Moderate
2    78.567762         Moderate
3    78.784284         Moderate
4    77.050971         Moderate
5    81.311056         Moderate
6    74.987407         Moderate
7    59.977625         Moderate
8    46.693894             Good
9    58.293548         Moderate
10   76.439837         Moderate
11  100.382710        Hazardous


In [55]:
print(r2_score(y['2022'],res['AQI']))

0.9233771594120797


In [56]:
print("Mean Absolute Error:",mean_absolute_error(y['2022'],res['AQI']))
print("Mean Squared Error:",mean_squared_error(y['2022'],res['AQI']))
print('Mean Absolute Percentage:',mean_absolute_percentage_error(y['2022'],res['AQI']))

Mean Absolute Error: 2.7964890049924684
Mean Squared Error: 21.765787197635646
Mean Absolute Percentage: 0.03653681608050171


In [57]:
khm_so2.index = pd.to_datetime(khm_so2['Year'].astype(str) + '-' + khm_so2['Month'], format='%Y-%b')
khm_so2.drop(['Year', 'Month'], axis=1, inplace=True)
khm_so2=khm_so2.sort_index()

In [58]:
khm_nox.index = pd.to_datetime(khm_nox['Year'].astype(str) + '-' + khm_nox['Month'], format='%Y-%b')
khm_nox.drop(['Year', 'Month'], axis=1, inplace=True)
khm_nox=khm_nox.sort_index()

In [59]:
khm_pm10.index = pd.to_datetime(khm_pm10['Year'].astype(str) + '-' + khm_pm10['Month'], format='%Y-%b')
khm_pm10.drop(['Year', 'Month'], axis=1, inplace=True)
khm_pm10=khm_pm10.sort_index()

In [60]:
#Performing exponential smoothing for the data to predict future dependent variables

from statsmodels.tsa.api import ExponentialSmoothing
model_so2= ExponentialSmoothing(khm_so2[:'2020'], trend='add', seasonal='mul')
fit_so2 = model_so2.fit(optimized=True)
fore_so2 = fit_so2.forecast(12)

from statsmodels.tsa.api import ExponentialSmoothing
model_nox= ExponentialSmoothing(khm_nox[:'2020'], trend='add', seasonal='mul')
fit_nox = model_nox.fit(optimized=True)
fore_nox = fit_nox.forecast(12)


from statsmodels.tsa.api import ExponentialSmoothing
model_pm10= ExponentialSmoothing(khm_pm10[:'2020'], trend='add', seasonal='mul')
fit_pm10 = model_pm10.fit(optimized=True)
fore_pm10 = fit_pm10.forecast(12)

c:\braindedmemory\Programming\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\braindedmemory\Programming\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\braindedmemory\Programming\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


In [61]:
#Accuracy
from sklearn.metrics import mean_absolute_percentage_error,mean_absolute_error
mae_so2 = mean_absolute_error(khm_so2['2021'],fore_so2['2021'] )
print('Mean Absolute Error for so2:', mae_so2)
mae_nox = mean_absolute_error(khm_nox['2022'],fore_nox )
print('Mean Absolute Error for nox:', mae_nox)
mae_pm10 = mean_absolute_error(khm_pm10['2022'],fore_pm10 )
print('MAPE for PM10:', mae_pm10)

Mean Absolute Error for so2: 0.7411120842387614
Mean Absolute Error for nox: 9.955428615318034
MAPE for PM10: 23.079740293360345


In [62]:
forecast_df=pd.DataFrame(fore_so2,columns=['SO2'])
forecast_df['NOX']=fore_so2
forecast_df['PM10']=fore_pm10
forecast_df

,SO2,NOX,PM10
2021-01-01,7.278958,7.278958,104.490942
2021-02-01,7.471522,7.471522,103.125841
2021-03-01,7.260689,7.260689,102.792048
2021-04-01,7.748837,7.748837,102.768610
2021-05-01,7.364020,7.364020,94.034818
2021-06-01,7.252586,7.252586,87.605934
2021-07-01,7.465636,7.465636,81.722741
2021-08-01,6.962766,6.962766,88.341431
2021-09-01,7.372979,7.372979,85.352399
2021-10-01,7.154087,7.154087,95.717477


In [63]:
final_pred=predictor(forecast_df)

          AQI Level of Concern
0   39.345255             Good
1   39.236406             Good
2   39.030407             Good
3   39.365747             Good
4   37.544837             Good
5   36.323983             Good
6   35.425857             Good
7   36.253218             Good
8   36.006935             Good
9   37.698082             Good
10  39.080706             Good
11  40.372043             Good


In [64]:
final_pred.to_csv('Khammam2023.csv')